In [185]:
import pandas as pd
import sqlite3

### Chinook Sample Database https://github.com/lerocha/chinook-database
### SQLite https://www.sqlite.org/
### SQLite Python https://docs.python.org/3/library/sqlite3.html

### Connect to the database

In [186]:
con = sqlite3.connect("/content/Chinook_Sqlite.sqlite")

### List tables

In [187]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,19,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Customer,Customer,386,CREATE TABLE [Customer]\n(\n [CustomerId] I...
2,table,Employee,Employee,392,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
3,table,Genre,Genre,395,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
4,table,Invoice,Invoice,396,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
5,table,InvoiceLine,InvoiceLine,399,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
6,table,MediaType,MediaType,402,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
7,table,Playlist,Playlist,404,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
8,table,PlaylistTrack,PlaylistTrack,405,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...
9,table,Track,Track,409,CREATE TABLE [Track]\n(\n [TrackId] INTEGER...


### Read the artist table into a dataframe

In [188]:
df = pd.read_sql_query("SELECT * FROM artist", con)

### show last rows of artist

In [189]:
df.tail()

,ArtistId,Name
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble
274,275,Philip Glass Ensemble
275,276,GCC Orchestra


### Add a new row to artist

In [190]:
df.loc[275] = ({'ArtistId': 276, 'Name': 'GCC Orchestra'})
# df = df.drop(275)
df.tail()

,ArtistId,Name
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble
274,275,Philip Glass Ensemble
275,276,GCC Orchestra


### Save the artist dataframe back to the database table

In [191]:
df.to_sql("Artist", con, if_exists="replace", index=False)

276

### Read the Playlist table into a dataframe

In [192]:
playlist = pd.read_sql_query("SELECT * FROM Playlist", con)
playlist.head()

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


### Read the PlaylistTrack table into a dataframe

In [193]:
track = pd.read_sql_query("SELECT * FROM PlaylistTrack", con)
track.head()

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


### Merge the track and playlist dataframes
https://www.w3schools.com/sql/sql_join_left.asp

https://pandas.pydata.org/docs/reference/api/pandas.merge.html

In [194]:
pd.merge(track, playlist, on='PlaylistId', how='left')

,PlaylistId,TrackId,Name
0,1,3402,Music
1,1,3389,Music
2,1,3390,Music
3,1,3391,Music
4,1,3392,Music
...,...,...,...
8710,17,2094,Heavy Metal Classic
8711,17,2095,Heavy Metal Classic
8712,17,2096,Heavy Metal Classic
8713,17,3290,Heavy Metal Classic


### groupby the merged dataframes and sort

In [195]:
pd.merge(track, playlist, on='PlaylistId', how='left').groupby('Name').count().sort_values(by='TrackId', ascending=False)


,PlaylistId,TrackId
Name,,
Music,6580,6580
90’s Music,1477,1477
TV Shows,426,426
Classical,75,75
Brazilian Music,39,39
Heavy Metal Classic,26,26
Classical 101 - Deep Cuts,25,25
Classical 101 - Next Steps,25,25
Classical 101 - The Basics,25,25


### Equivalent SQL join query

In [196]:
sql = '''select p.Name , count(*) as Tracks
      from PlaylistTrack as pt
      left join playlist as p on p.PlaylistId = pt.PlaylistId
      group by p.Name
      order by Tracks desc'''
top_playlists = pd.read_sql_query(sql, con)
top_playlists

,Name,Tracks
0,Music,6580
1,90’s Music,1477
2,TV Shows,426
3,Classical,75
4,Brazilian Music,39
5,Heavy Metal Classic,26
6,Classical 101 - The Basics,25
7,Classical 101 - Next Steps,25
8,Classical 101 - Deep Cuts,25
9,Grunge,15


### Save the results of the query into a new table

In [197]:
top_playlists.to_sql("top_playlists", con, if_exists="replace")
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,19,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Customer,Customer,386,CREATE TABLE [Customer]\n(\n [CustomerId] I...
2,table,Employee,Employee,392,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
3,table,Genre,Genre,395,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
4,table,Invoice,Invoice,396,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
5,table,InvoiceLine,InvoiceLine,399,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
6,table,MediaType,MediaType,402,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
7,table,Playlist,Playlist,404,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
8,table,PlaylistTrack,PlaylistTrack,405,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...
9,table,Track,Track,409,CREATE TABLE [Track]\n(\n [TrackId] INTEGER...


### Drop the newly created table

In [198]:
sql = '''drop table top_playlists'''
con.execute(sql)

### List tables

In [199]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)
tables

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,19,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Customer,Customer,386,CREATE TABLE [Customer]\n(\n [CustomerId] I...
2,table,Employee,Employee,392,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
3,table,Genre,Genre,395,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
4,table,Invoice,Invoice,396,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
5,table,InvoiceLine,InvoiceLine,399,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
6,table,MediaType,MediaType,402,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
7,table,Playlist,Playlist,404,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
8,table,PlaylistTrack,PlaylistTrack,405,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...
9,table,Track,Track,409,CREATE TABLE [Track]\n(\n [TrackId] INTEGER...


### Close the database connection

In [200]:
con.close()